In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import threading
import multiprocessing
import random
from time import sleep
from time import time
import scipy.signal
from env import stock
import collections as col

In [ ]:
class Q_network:
    def __init__(self,in_size,out_size,opt,layers=4,width=8, train=True):
        self.inputs = tf.placeholder(shape=[None, in_size], dtype=tf.float32)
        if layers > 0:
            self.a = [tf.layers.dense(self.inputs, width, activation=tf.nn.relu)]
            self.v = [tf.layers.dense(self.inputs, width, activation=tf.nn.relu)]
            for i in range(layers-1):
                self.a.append(tf.layers.dense(self.a[-1], width, activation=tf.nn.relu))
                self.v.append(tf.layers.dense(self.v[-1], width, activation=tf.nn.relu))
                
                self.a.append(tf.layers.dense(self.a[-1], out_size, activation=None))
                self.v.append(tf.layers.dense(self.v[-1], out_size, activation=None))

        else:
            self.a = [tf.layers.dense(self.inputs, out_size, activation=None)]
            self.v = [tf.layers.dense(self.inputs, 1, activation=None)]
        
        self.guessQ = self.v[-1] + tf.subtract(self.a[-1], tf.reduce_mean(self.a[-1],axis=1,keep_dims=True))
        self.choice = tf.argmax(self.guessQ, 1)
        
        if train:
            self.o = tf.train.AdamOptimizer(learning_rate=opt)
            self.true_Q = tf.placeholder(shape=[None], dtype=tf.float32)
            self.action = tf.placeholder(shape=[None], dtype=tf.uint8)
            self.hot = tf.one_hot(self.action, 3)
            
            self.Q = tf.reduce_sum(tf.multiply(self.guessQ, self.hot), axis=1)
            
            #local_vars = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES)
            self.err = tf.square(self.true_Q - self.Q)
            self.loss = tf.reduce_mean(self.err)
            self.grads = self.o.compute_gradients(self.loss)
            self.train_op = self.o.apply_gradients(self.grads)

In [ ]:
class memories:
    def __init__(self, buffer_size=100000):
        self.buffer = []
        self.buffer_size = buffer_size
    def add_memory(self, memory):
        self.buffer.extend(memory)
    def retrieve_memories(self, num):
        ret = np.array(random.sample(self.buffer,num))
        #print(ret.shape)
        #print(ret)
        return ret

In [ ]:
def updateTargetGraph(tfVars,tau):
    total_vars = len(tfVars)
    op_holder = []
    for idx,var in enumerate(tfVars[0:total_vars//2]):
        op_holder.append(tfVars[idx+total_vars//2].assign((var.value()*tau) + ((1-tau)*tfVars[idx+total_vars//2].value())))
    return op_holder

def updateTarget(op_holder,sess):
    for op in op_holder:
        sess.run(op)

In [ ]:
stock = stock()
o = 1e-3
Q_main = Q_network(14, 3, o)
Q_target = Q_network(14, 3, o)

e = 1
stop = .001
decay = (e - stop)/10000
n_eps = 50000
training_steps = 8
step_count = 0
group_count = 0
update_steps = 4
batch_size = 32
discount = .99
path = './graph/saved'
membuf = memories()
saver = tf.train.Saver()
restore = False

trainables = tf.trainable_variables()
target_ops = updateTargetGraph(trainables,.001)


rewards = []
wl_rate = []
err = []

with tf.Session() as sess:
    if restore:
        ckpt = tf.train.get_checkpoint_state(path)
        saver.restore(sess,ckpt.model_checkpoint_path)
    else:
        sess.run(tf.global_variables_initializer())
    for i in range(n_eps):
        stock.get_new_day()
        stock.get_first_ohlc()
        inp = stock.next_data()
        epbuf = memories()
        total_reward = 0
        count_moves = 0
        count_wins = 0
        while len(inp) is not 0:
            if np.random.rand(1) < e or step_count < training_steps:
                a = np.random.randint(0,2)
            else:
                a = sess.run(Q_main.choice, feed_dict={Q_main.inputs:[inp]}) #fix
            r = stock.make_choice(a)
            if (r is not 0):
                count_moves += 1
                if (r > 0):
                    count_wins += 1
            step_count += 1
            inp2 = stock.next_data()
            if (len(inp2) is not 0):
                epbuf.add_memory(np.reshape(np.array([inp,a,r,inp2]),[1,4]))
            total_reward += r
            if group_count > training_steps:
                if e > stop:
                    e = e-decay
                if (step_count % update_steps) == 0:
                    training_batch = membuf.retrieve_memories(batch_size)
                    #print(training_batch[:,3])
                    Q_val_1 = sess.run(Q_main.choice, feed_dict={Q_main.inputs:np.vstack(training_batch[:,3])})
                    Q_val_2 = sess.run(Q_target.guessQ, feed_dict={Q_target.inputs:np.vstack(training_batch[:,3])})
                    
                    D_Q = Q_val_2[range(batch_size), Q_val_1]
                    target_Q = training_batch[:,2]+(discount*D_Q)
                    #print(np.vstack(training_batch[:,1]))
                    fd = {Q_main.inputs:np.vstack(training_batch[:,0]),
                          Q_main.true_Q:target_Q,
                          Q_main.action:np.reshape(np.vstack(training_batch[:,1]),[batch_size])}
                    train = sess.run([Q_main.grads,Q_main.train_op,Q_main.loss], 
                                     feed_dict=fd)
                    err.append(train[2])
                    updateTarget(target_ops,sess)
            inp = inp2
        group_count += 1
        membuf.add_memory(epbuf.buffer)
        rewards.append(total_reward)
        if (count_moves is 0):
            wl_rate.append(0)
        else:
            wl_rate.append(count_wins/count_moves)
        if(i % 8 == 0):
            saver.save(sess, path)
            print(np.mean(rewards[-8:]))
            print(np.mean(wl_rate[-8:]))
            print(np.mean(err[-8:]))
            print("")

This is a Q-network that is made to try to learn to buy and sell stocks with reinforcement learning. It ultimately failed.